In [1]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, func

from flask import Flask, jsonify


In [2]:
# Database Setup: importing dataframes and CSV files
#################################################
engine = create_engine("sqlite:///hawaii.sqlite")
df = pd.read_csv('hawaii_measurements.csv')
stations_df = pd.read_csv('hawaii_stations.csv')
total_stations = stations_df['station'].nunique()

In [3]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

# Create our session (link) from Python to the DB

session = Session(engine)


C:\Users\nickn\AppData\Local\Temp\ipykernel_12860\2597355465.py:4: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Base.prepare(engine, reflect=True)


In [4]:
#################################################
# Flask Setup
#################################################

app = Flask(__name__)

In [5]:
#################################################
# Flask Routes
#################################################
@app.route('/')
def home():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start>/<end>"
    )

@app.route('/api/v1.0/precipitation')
def precipitation():
    # Convert the query results to a dictionary using date as the key and prcp as the value
    prcp_dict = pd.Series(df.prcp.values, index=df.date).to_dict()
    return jsonify(prcp_dict)

@app.route('/api/v1.0/stations')
def stations():
    # Return a JSON list of stations from the dataset
    stations_list = stations_df['station'].tolist()
    return jsonify(stations_list)

@app.route('/api/v1.0/tobs')
def tobs():
    # Query the dates and temperature observations of the most-active station for the last year of data
    # Assuming 'most_active_station' is the ID of the most active station
    # and 'last_year_data' is a dataframe with the last year of data for this station
    tobs_data = last_year_data[['date', 'tobs']].set_index('date').to_dict()
    return jsonify(tobs_data)

@app.route('/api/v1.0/<start>')
def start(start):
    # Return a JSON list of the minimum temperature, the average temperature, and the max temperature for a given start
    # Assuming 'calculate_temps' is a function that calculates the TMIN, TAVG, and TMAX
    temps = calculate_temps(start_date=start)
    return jsonify(temps)

@app.route('/api/v1.0/<start>/<end>')
def start_end(start, end):
    # Return a JSON list of the minimum temperature, the average temperature, and the max temperature for a given start-end range
    # Assuming 'calculate_temps' is a function that calculates the TMIN, TAVG, and TMAX
    temps = calculate_temps(start_date=start, end_date=end)
    return jsonify(temps)

def calculate_temps(start_date, end_date=None):
    # Filter the data based on the provided dates
    if end_date:
        
        temps = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    else:
        # If no end date is provided, filter from the start date onwards
        temps = df[df['date'] >= start_date]
    
    # Calculate the minimum, average, and maximum temperatures
    tmin = temps['tobs'].min()
    tavg = temps['tobs'].mean()
    tmax = temps['tobs'].max()
    
    # Return the calculated temperatures
    return {'TMIN': tmin, 'TAVG': tavg, 'TMAX': tmax}

   

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\nickn\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
